In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import isaacgym
import isaacgymenvs
from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, get_rlgames_env_creator

from rl_games.common import env_configurations, vecenv
from rl_games.torch_runner import Runner
from rl_games.algos_torch import model_builder

from omegaconf import DictConfig, OmegaConf

import torch
import numpy as np
import matplotlib.pyplot as plt

from isaacgymenvs.qp.fista import ForceQP, LocationQP, FISTA
from isaacgymenvs.qp.qp_utils import *
from isaacgymenvs.qp.vecrobotics import *

In [ ]:
cfg = OmegaConf.load("../isaacgymenvs/cfg/config.yaml")
cfg.task_name = "TrifingerNYU"
cfg.num_envs = 4
cfg.task = OmegaConf.load("../isaacgymenvs/cfg/task/TrifingerNYU.yaml")

cfg.task.env.command_mode = "torque"
cfg.task.env.reset_distribution.object_initial_state.type = "default"
cfg.task.normalize_action = False
cfg.headless = False

In [ ]:
def create_env_thunk(**kwargs):
    envs = isaacgymenvs.make(
            cfg.seed, 
            cfg.task_name, 
            cfg.task.env.numEnvs, 
            cfg.sim_device,
            cfg.rl_device,
            cfg.graphics_device_id,
            cfg.headless,
            cfg.multi_gpu,
            cfg.capture_video,
            cfg.force_render,
            cfg,
            **kwargs,
        )
    return envs

In [ ]:
lifting_data = np.load("data/lifting2.npz", allow_pickle=True)["data"]

In [ ]:
device = cfg.sim_device
envs = create_env_thunk()

In [ ]:
def get_force_qp_data(ftip_pos: torch.Tensor, object_pose: torch.Tensor, 
                      total_force_des: torch.Tensor, torque_ref: torch.Tensor, jacobian: torch.Tensor,
                      weights: List[float]):
    # get ftip positin in the object frame
    batch_size, num_ftip, _ = ftip_pos.shape
    num_vars = num_ftip * 3

    p = SE3_inverse_transform(object_pose.repeat_interleave(3, dim=0), ftip_pos.view(-1, 3))
    contact_normals = get_cube_contact_normals(p)
    object_orn = quat2mat(object_pose[:, 3:])
    jacobian_transpose = torch.transpose(jacobian, 1, 2)
    
    
    # force cost
    R = get_contact_frame_orn(contact_normals)
    R_vstacked = R.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
    Q1 = R_vstacked @ R_vstacked.transpose(1, 2)
    total_force_des_object_frame = bmv(object_orn.transpose(1,2), total_force_des)
    q1 = -2 * bmv(R_vstacked, total_force_des_object_frame)
    
    # torque cost
    pxR = vec2skewsym_mat(p) @ R
    pxR_vstacked = pxR.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
    Q2 = pxR_vstacked @ pxR_vstacked.transpose(1, 2)
    
    # joint torque cost
    R_reshaped = R.view(batch_size, num_ftip, 3, 3)
    R_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
    for i in range(num_ftip):
        R_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = R_reshaped[:, i]
    object_orn_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
    for i in range(num_ftip):
        object_orn_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = object_orn
    A = jacobian_transpose @ object_orn_diag @ R_diag
    Q3 = torch.transpose(A, 1, 2) @ A
    q3 = -2 * bmv(A, torque_ref)
    
    # regularization
    Q4 = 1e-4 * torch.eye(3 * num_ftip).repeat(batch_size, 1, 1).to(ftip_pos.device)
    
    # If any of contact normals == 0, set the desired force and torque weights to zero
    reshaped_tensor = Q1.view(batch_size, -1)
    mask = torch.any(reshaped_tensor[:, ::num_vars+1] == 0, dim=1)

    # construct total cost
    weights = torch.tensor(weights, dtype=torch.float32, device=ftip_pos.device).repeat(batch_size, 1)
    w1, w2, w3 = weights.split(1, dim=1)
    w1[mask] = 0
    w2[mask] = 0
    w3[mask] = 0
    
    q = w1*q1 + w3*q3
    
    w1 = w1.view(batch_size, 1, 1)
    w2 = w2.view(batch_size, 1, 1)
    w3 = w3.view(batch_size, 1, 1)
    
    Q = w1 * Q1 + w2 * Q2 + w3 * Q3 + Q4
    
    return Q, q, R_vstacked, pxR_vstacked, contact_normals

In [ ]:
# get ftip states
envs.reset_idx(torch.arange(cfg.num_envs))
N = 700
action_buffer = torch.zeros(N, 9).to(device)
projected_torque_buffer = torch.zeros(N, 9).to(device)

ftip_pos_buffer = torch.zeros(N, 3, 3).to(device)
ftip_pos_local_buffer = torch.zeros(N, 3, 3).to(device)
object_pose_buffer = torch.zeros(N, 7).to(device)
contact_normals_buffer = []

q_buffer = torch.zeros(N, 9).to(device)
dq_buffer = torch.zeros(N, 9).to(device)
obs, rwds, resets, info = envs.step(torch.zeros(cfg.num_envs, 9))

In [ ]:
# construct force QP
batch_size = cfg.num_envs
num_vars = 9
force_lb = -1.5 * torch.ones(batch_size, num_vars)
force_ub = 1.5 * torch.ones(batch_size, num_vars)

force_qp = ForceQP(batch_size, num_vars, friction_coeff=1.0, device=device)
force_qp_solver = FISTA(force_qp, device=device)
max_it = 50

for n in range(N):
    q = envs._dof_position
    dq = envs._dof_velocity
    q_buffer[n] = q[0]
    dq_buffer[n] = dq[0]
    
    ftip_state = envs._rigid_body_state[:, envs._fingertip_indices]
    ftip_pos = ftip_state[:, :, 0:3]
    ftip_vel = ftip_state[:, :, 7:10]
    ftip_pos_buffer[n] = ftip_pos[0]
    
    object_pose = envs._object_state_history[0][:, 0:7]
    object_position = object_pose[:, 0:3]
    desired_object_position = envs._desired_object_poses_buf[:, 0:3]
    object_orn = quat2mat(object_pose[:, 3:])
    object_pose_buffer[n] = object_pose[0]
    
    time_idx = 0 if n < 200 else 20 * (n - 200)
    ftip_pos_des = torch.tensor(lifting_data[time_idx]['policy']['controller']['ft_pos_des'], dtype=torch.float32).to(device)
    ftip_vel_des = torch.tensor(lifting_data[time_idx]['policy']['controller']['ft_vel_des'], dtype=torch.float32).to(device)

    ftip_pos_diff = ftip_pos_des - ftip_pos.reshape(cfg.num_envs, 9)
    ftip_vel_diff = ftip_vel_des - ftip_vel.reshape(cfg.num_envs, 9)
    
    task_space_force = torch.tensor([200,200,200]*3, dtype=torch.float32, device=envs.device) * ftip_pos_diff
    task_space_force += torch.tensor([5]*9, dtype=torch.float32, device=envs.device)  * ftip_vel_diff
    
    jacobian = envs.get_fingertip_jacobian_linear()
    jacobian_transpose = torch.transpose(jacobian, 1, 2)
    
#     action = -0.36 + 0.72 * torch.rand(cfg.num_envs, 9).to(device)
#     task_space_force = -1.5 + 3. * torch.rand(cfg.num_envs, 9).to(device)
    
    torque_ref = bmv(jacobian_transpose, task_space_force)
    
    desired_object_acceleration = 10 * (desired_object_position - object_position)
    total_force_des = 0.08 * desired_object_acceleration
    force_qp_cost_weights = [1, 200, 0.01]
    Q, q, R_vstacked, pxR_vstacked, contact_normals = get_force_qp_data(ftip_pos, 
                                                                        object_pose,
                                                                        total_force_des,
                                                                        torque_ref, 
                                                                        jacobian,
                                                                        force_qp_cost_weights)
    
    force_qp_solver.prob.set_data(Q, q, force_lb, force_ub)
    force_qp_solver.reset()
    for i in range(max_it):
        force_qp_solver.step()
    ftip_force_contact_frame = force_qp_solver.prob.yk.clone()

    # convert force to the world frame
    R = R_vstacked.reshape(-1, 3, 3).transpose(1, 2)
    ftip_force_object_frame = stacked_bmv(R, ftip_force_contact_frame)
    object_orn = quat2mat(object_pose[:, 3:7]).repeat(3, 1, 1)
    task_space_force2 = stacked_bmv(object_orn, ftip_force_object_frame)
    
    action = torque_ref + bmv(jacobian_transpose, task_space_force2)
    
    action_buffer[n] = action[0]
#     projected_torque_buffer[n] = projected_torque[0]
    obs, rwds, resets, info = envs.step(action)

In [ ]:
# weights = force_qp_cost_weights
# # get ftip positin in the object frame
# batch_size, num_ftip, _ = ftip_pos.shape
# num_vars = num_ftip * 3

# p = SE3_inverse_transform(object_pose.repeat_interleave(3, dim=0), ftip_pos.view(-1, 3))
# contact_normals = get_cube_contact_normals(p)
# object_orn = quat2mat(object_pose[:, 3:])
# jacobian_transpose = torch.transpose(jacobian, 1, 2)


# # force cost
# R = get_contact_frame_orn(contact_normals)
# R_vstacked = R.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
# Q1 = R_vstacked @ R_vstacked.transpose(1, 2)
# total_force_des_object_frame = bmv(object_orn.transpose(1,2), total_force_des)
# q1 = -2 * bmv(R_vstacked, total_force_des_object_frame)

# # torque cost
# pxR = vec2skewsym_mat(p) @ R
# pxR_vstacked = pxR.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
# Q2 = pxR_vstacked @ pxR_vstacked.transpose(1, 2)

# # joint torque cost
# R_reshaped = R.view(batch_size, num_ftip, 3, 3)
# R_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
# for i in range(num_ftip):
#     R_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = R_reshaped[:, i]
# object_orn_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
# for i in range(num_ftip):
#     object_orn_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = object_orn
# A = jacobian_transpose @ object_orn_diag @ R_diag
# Q3 = torch.transpose(A, 1, 2) @ A
# q3 = -2 * bmv(A, action)

# # regularization
# Q4 = 1e-4 * torch.eye(3 * num_ftip).repeat(batch_size, 1, 1).to(ftip_pos.device)

# # If any of contact normals == 0, set the desired force and torque weights to zero
# reshaped_tensor = Q1.view(batch_size, -1)
# mask = torch.any(reshaped_tensor[:, ::num_vars+1] == 0, dim=1)

# # construct total cost
# weights = torch.tensor(weights, dtype=torch.float32, device=ftip_pos.device).repeat(batch_size, 1)
# w1, w2, w3 = weights.split(1, dim=1)
# w1[mask] = 0
# w2[mask] = 0
# w3[mask] = 0

# q = w1*q1 + w3*q3

# w1 = w1.view(batch_size, 1, 1)
# w2 = w2.view(batch_size, 1, 1)
# w3 = w3.view(batch_size, 1, 1)

# Q = w1 * Q1 + w2 * Q2 + w3 * Q3 + Q4

In [ ]:
task_space_force[:, :3] + task_space_force[:, 3:6] + task_space_force[:, 6:]

In [ ]:
task_space_force2[:, :3] + task_space_force2[:, 3:6] + task_space_force2[:, 6:]

In [ ]:
total_force_des

In [ ]:
torque_ref

In [ ]:
action